# MAIN FUNCTIONS NOTEBOOK
### Project: CRYPTO-TRADING-SYSTEM
### Copyright: Mantis Trading
#### Table of Contents
    1. Data Pre-Processing Functions
    2. Feature Engineering Functions
    3. XG Boost Mult-Classification Functions
    4. Backtesting Functions (backtest, performance viz)
    5. Live Trade Functions 
    6. Real Time Management Functions (Dash, Reporting) 
    7. Libraries
    8. Appendix
    

##### WORK ITEMS
    1. update test dataframe so every 6 hrs there is a row... forward fill data (team)
    2. Re-Train Model on Sequential Data, Test on sequential Data and output predicted df for test(70/30 split)  (team)
    3. Finish Strategy logic in backtest (grant)
    4. Finish performance calculations and visualations code for backtest (scott)
    5. Finish live_trader_mantis.py (team)
        - finish initialization function
        - finish fetch_data function
        - finish generate_feature function
        - finish rebalancePortfolio function
        - add shrimpy websocket code
        - add shrimpy execution functionality
    6. Finish real_time_manager.py (team)
        - finish performance_dash function
        - finish send_alerts function
        - finish logging function

Improvements
    
    - calculate more features for modeling
    - grid search and shapley for feature engineering
    - Test different classification models
    - optimize for bucket thresholds 
    -
---

### 1. Data Pre-Processing Functions

In [ ]:
# Get exchange assets and Trading Pairs from Shrimpy API
def get_base_exchange_data(exchange):
    # Get Digital Asset Data
    def get_exchange_assets(exchange):
        exchange_assets = shrimpy_client.get_exchange_assets(exchange)
        exchange_assets_df = pd.DataFrame(columns=['id','name', 'symbol', 'trading_symbol'])
        for key, value in enumerate(exchange_assets):
            exchange_assets_df.loc[key] = [value['id'], value['name'], value['symbol'], value['tradingSymbol']]
        return exchange_assets_df

    def calc_trading_pairs_df(exchange):
        exchange_pairs = shrimpy_client.get_trading_pairs(exchange)
        exchange_pairs_df = pd.DataFrame(columns=['base','quote'])
        for key, value in enumerate(exchange_pairs):
            exchange_pairs_df.loc[key] = [value['baseTradingSymbol'],value['quoteTradingSymbol']]
        return exchange_pairs_df
    return get_exchange_assets(exchange), calc_trading_pairs_df(exchange)


In [ ]:
# Get and organize closing prices into datafram for one ticker from shrimpy rest api
def get_prices_df(candles):
    time = []
    prices = []
    for key, value in enumerate(candles):
        time.append(value['time'])
        prices.append(value['close'])
    prices_df = pd.DataFrame(list(zip(time, prices)), columns = ['time', 'close'])
    prices_df['time'] = pd.to_datetime(prices_df['time'])
    prices_df['close'] = pd.to_numeric(prices_df['close'])
    prices_df.set_index('time', inplace=True)
    return prices_df

In [ ]:
# Get and organize closing prices into dataframe from list of trading pairs from one exchange from shrimpy rest api
def get_prices(exchange, trading_pairs_df, interval, start):
    master_prices_df = pd.DataFrame()
    for index, row in trading_pairs_df.iterrows():
        candles = shrimpy_client.get_candles(exchange, row['base'], row['quote'], interval, start)
        time = []
        prices = []
        for key, value in enumerate(candles):
            time.append(value['time'])
            prices.append(value['close'])
        prices_df = pd.DataFrame(list(zip(time, prices)), columns = ['time', row['base'] + "_" + row['quote']])
        prices_df['time'] = pd.to_datetime(prices_df['time'])
        if master_prices_df.empty:
            master_prices_df = prices_df
        else:
            master_prices_df = pd.merge(master_prices_df, prices_df, left_on='time', right_on = 'time', how = 'left')
    return master_prices_df

---

### 2. Feature Engineering Functions

In [ ]:
# make sure rows are every 6hrs, if there is no row-then make one and forward fill data (shrimpy doesn't print candle if there is no tick)
def calc_feature_dataframe(prices_df):
    ## cumulative returns as velocity
    ## Log returns as velocity
    ## partials?
    ## Lags?
    ## Technical Indicators
    ## Social Indicators
    df_features = prices_df
    # Construct dependent variable
    df_features['returns'] = df_features['close'].pct_change()
    # Calculate cumulative returns
    df_features['cum_returns'] = (df_features['returns']+1).cumprod()
    # ----------------------- Price Dynamics --------------------------------
    # price dynamics as a one Dimensional particle problem in physics
    df_features['price_velocity_2'] = df_features['close'].pct_change(2)
    df_features['price_velocity_3'] = df_features['close'].pct_change(3)
    df_features['price_velocity_4'] = df_features['close'].pct_change(4)
    df_features['price_velocity_7'] = df_features['close'].pct_change(7)
    df_features['price_velocity_30'] = df_features['close'].pct_change(30)
    
    df_features['price_acceleration_1'] = df_features['returns'].pct_change(1)
    df_features['price_acceleration_2'] = df_features['price_velocity_2'].pct_change(2)
    df_features['price_acceleration_3'] = df_features['price_velocity_3'].pct_change(3)
    df_features['price_acceleration_4'] = df_features['price_velocity_4'].pct_change(4)
    df_features['price_acceleration_7'] = df_features['price_velocity_7'].pct_change(7)
    df_features['price_acceleration_30'] = df_features['price_velocity_30'].pct_change(30)

    df_features['rolling_mean_velocity_2'] = df_features['returns'].rolling(window=2).mean()
    df_features['rolling_mean_velocity_3'] = df_features['returns'].rolling(window=3).mean()
    df_features['rolling_mean_velocity_4'] = df_features['returns'].rolling(window=4).mean()
    df_features['rolling_mean_velocity_7'] = df_features['returns'].rolling(window=7).mean()
    df_features['rolling_mean_velocity_30'] = df_features['returns'].rolling(window=30).mean()
    
    df_features['rolling_mean_acceleration_2'] = df_features['price_acceleration_1'].rolling(window=2).mean()
    df_features['rolling_mean_acceleration_3'] = df_features['price_acceleration_1'].rolling(window=3).mean()
    df_features['rolling_mean_acceleration_4'] = df_features['price_acceleration_1'].rolling(window=4).mean()
    df_features['rolling_mean_acceleration_7'] = df_features['price_acceleration_1'].rolling(window=7).mean()
    df_features['rolling_mean_acceleration_30'] = df_features['price_acceleration_1'].rolling(window=30).mean()
    # To extend space to entire line the log price is mapped to position x(t) in the space by
    # x(t) = log(S(t))   where S(t) is the price of the instrument
    df_features['log_price'] = np.log(df_features['close'])
    df_features['log_returns'] = df_features['log_price'].diff() # Diff or percent change
    #df_features['log_return_pct']  = df_features['log_price'].pct()
    #df_features['cum_log_returns'] =(df_features['log_returns_pct'] + 1).cumprod()
    # Assumption: Returns of financial instruments are lognormally distributed
    # v(t) = R(t) = dx(t)/dt where v(t) is the velocity of the instrument in the log price space, x(t)
    
    # ------------------------------ partial price dynamics ---------------------
    # -------------------------------Technical Indicators ------------------------
    df_features.dropna(inplace=True)
    return df_features

---

### 3. XG Boost Multi-Classification Functions

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn import svm
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Not finished
def xg_boost_multiclassifier(df):
    #bin_labels_5_name = ['very_bear', 'bear', 'neutral', 'bull', 'very_bull']
    bin_labels_5 = ['1', '2', '3', '4', '5']
    class_df = df.copy()
    class_df['return_class'] = pd.cut(df_features.returns,
                                  bins=[-np.inf, -.02,-.0025, .0025, .02, np.inf], labels=bin_labels_5)
    class_df.dropna(inplace=True)
    # Need to clean up numbers for model
    cleanup_nums = {"return_class": {"1": round(1), "2": round(2), "3": round(3), "4": round(4), "5": round(5)}}
    class_df.replace(cleanup_nums, inplace=True)
    # X,y Test/Train Split for ML Classification Models
    X = class_df.copy()
    X.drop(["close", "returns", "return_class"], axis=1, inplace=True)
    y = class_df["return_class"].values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Train Model
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=5).fit(X_train,y_train)
    # Test model
    predictions = model.predict(X_test)
    predicted_returns = predictions.reshape(-1,1)
    real_returns = y_test.reshape(-1, 1)
    # Create dataframe with columns real returns, predicted returns, and accuracy
    predicted_returns_xgb = pd.DataFrame({
        "Real": real_returns.ravel(),
        "y_test": y_test.ravel(),
        "Predicted": predicted_returns.ravel()
    })
    return predicted_returns_xgb

- need to make sure timestamps are on predicted dataframe for backtest
- need to train sequentially
- save model using joblib

---

### 4. Backtesting Functions

In [ ]:
def return_analysis(test_df, initial_capital, exchange, rebalance_freq, start, end, fee_perc, tax_fee):
    name_of_model = 'XGB Multi-Classifier'
    initial_capital = float(initial_capital)
    # BTC-USD Trend thresholds
    very_bullish = 5 # if btc price prediction is greater than 
    bullish = 4 # if btc
    nuetral = 3 # if btc is predicting 0 returns
    bearish = 2 # if btc is predicting negative returns
    very_bearish = 1 # if btci predicting very negative returns
    # Set Threshold Weights (create global variables as input...)
    # ***** figure out how to train a model to predict class ranges for trend thresholds
    # ***** figure out how to train a model to predict best allocation wts given the class prediction
    btc_wts = [.2,.4,.5,.5,.4]
    usd_wts = [.8,.6,.4,.2, 0]
    alt_wts = [0,0,.1,.3,.6]
    # Create balance lists to populate when looping through dataframe
    bucket_1_position = []
    bucket_2_position = []
    bucket_3_position = []
    usd_balance = []
    btc_balance = []
    time = []
    for index, row in test_df.iterrows():
        # Strategy Logic    
        if row['predicted_class'] == 5: # Very Bull
            bucket_1_position.append(usd_wts[4])
            bucket_2_position.append(btc_wts[4])
            bucket_3_position.append(alt_wts[4])
            balance_usd = row['']* usd_wts[4]
            balance_btc = row['']
            
            time.append(row['time'])
        elif row['predicted_class'] == 4: # Bull
            bucket_1_position.append(usd_wts[3])
            bucket_2_position.append(btc_wts[3])
            bucket_3_position.append(alt_wts[3])
            time.append(row['time'])          
        elif row['predicted_class'] == 3: # Neutral
            bucket_1_position.append(usd_wts[2])
            bucket_2_position.append(btc_wts[2])
            bucket_3_position.append(alt_wts[2])
            time.append(row['time'])
        elif row['predicted_class'] == 2: # Bear
            bucket_1_position.append(usd_wts[1])
            bucket_2_position.append(btc_wts[1])
            bucket_3_position.append(alt_wts[1])
            time.append(row['time'])
        elif row['predicted_class'] == 1: # Very Bear
            bucket_1_position.append(usd_wts[0])
            bucket_2_position.append(btc_wts[0])
            bucket_3_position.append(alt_wts[0])
            time.append(row['time'])

    # Create positions dataframe  
    positions_df = pd.DataFrame(list(zip(time, bucket_1_position, bucket_2_position, bucket_3_position)),
                                 columns =['time', 'bucket_1_position', 'bucket_2_position', 'bucket_3_position'])   
    
    # Merge positions dataframe with backtest dataframe
    final_backtest_df = pd.merge(test_df, positions_df, how='left', on='time')
    # Market Dynamics Columns
    final_backtest_df['btc_price_change'] = backtest_df['btc_price'].pct_change()
    final_backtest_df['eth_price_change'] = backtest_df['ethbtc_price'].pct_change()
    # Portfolio Columns
    final_backtest_df['portfolio_balance_usd'] = final_backtest_df['bucket_1_position'] 
    final_backtest_df['portfolio_returns_usd'] =  final
    final_backtest_df['portfolio_cum_returns_usd'] = 
    final_backtest_df['portfolio_balance_btc'] = 
    final_backtest_df['portfolio_returns_btc'] =
    final_backtest_df['portfolio_cum_returns_btc'] =
    # Position Columns
    final_backtest_df['bucket_1_position_change'] = backtest_df['bucket_1_position'].diff()
    final_backtest_df['bucket_2_position_change'] = backtest_df['bucket_2_position'].diff()
    final_backtest_df['bucket_3_position_change'] = backtest_df['bucket_3_position'].diff()
    
    
    
    # ---------------------------------- Calc Portfolio Performance ------------------------
    
    #backtest_results_df1['alpha'] = backtest_results_df1['usd_cum_returns'] - backtest_results_df1['btcusd_cum_returns']
    #backtest_results_df1['usd_rolling_returns_wk'] = backtest_results_df1['usd_perc_change'].rolling(7).sum()
    #backtest_results_df1['btc_rolling_returns_wk'] = backtest_results_df1['btc_perc_change'].rolling(7).sum()
    #backtest_results_df1['usd_rolling_vol_wk'] = backtest_results_df1['usd_perc_change'].rolling(7).std()
    #backtest_results_df1['btc_rolling_vol_wk'] = backtest_results_df1['btc_perc_change'].rolling(7).std()
    #backtest_results_df1['usd_rolling_returns_mo'] = backtest_results_df1['usd_perc_change'].rolling(7).sum()
    #backtest_results_df1['btc_rolling_returns_mo'] = backtest_results_df1['btc_perc_change'].rolling(7).sum()
    #backtest_results_df1['usd_rolling_vol_mo'] = backtest_results_df1['usd_perc_change'].rolling(30).std()
    #backtest_results_df1['btc_rolling_vol_mo'] = backtest_results_df1['btc_perc_change'].rolling(30).std()
    #
    #backtest_results_df1['usd_rolling_sharpe_wk'] = backtest_results_df1['usd_rolling_returns_wk']/backtest_results_df1['usd_rolling_vol_wk']
    #backtest_results_df1['btc_rolling_sharpe_wk'] = backtest_results_df1['btc_rolling_returns_wk']/backtest_results_df1['btc_rolling_vol_wk']
    #backtest_results_df1['usd_rolling_sharpe_mo'] = backtest_results_df1['usd_rolling_returns_mo']/backtest_results_df1['usd_rolling_vol_mo']
    #backtest_results_df1['btc_rolling_sharpe_mo'] = backtest_results_df1['btc_rolling_returns_mo']/backtest_results_df1['btc_rolling_vol_mo']
    
    # More Risk Measures
        # Max Drawdown 
        # Skewness
        # Kurtosis
        # VAR
        # CVAR
        # Time under water   
    usd_metrics= [
        'initial_capital',
        'cumulative_returns',
        'annualized_returns',
        'mthly_returns',
        'annual_vol',
        'mthly_vol',
        'annual_sharpe',
        'mthly_sharpe',
        'annual_sortino',
        'mthly_sortino',
    ]
    
    btc_metrics = [
        
        'initial_capital',
        'cumulative_returns',
        'annualized_returns',
        'mthly_returns',
        'annual_vol',
        'mthly_vol',
        'annual_sharpe',
        'mthly_sharpe',
        'annual_sortino',
        'mthly_sortino',
    ]
    
    columns = ['Backtest']
    
    port_eval_df_usd = pd.DataFrame(index=usd_metrics, columns=columns)
    
    port_eval_df_usd.loc['initial_capital'] = initial_capital
    port_eval_df_usd.loc['cumulative_returns'] = final_backtest_df['portfolio_cum_returns_usd'].iloc[-1]
    port_eval_df_usd.loc['annualized_returns'] = final_backtest_df['portfolio_returns_usd'].mean() *365  #Crypto trades 365 days per year
    port_eval_df_usd.loc['mthly_returns'] = final_backtest_df['portfolio_returns_usd'].mean() *30
    port_eval_df_usd.loc['annual_vol'] = final_backtest_df['portfolio_returns_usd'].std() * np.sqrt(365)  #Crypto trades 365 days per year
    port_eval_df_usd.loc['mthly_vol'] = final_backtest_df['portfolio_returns_usd'].std() * np.sqrt(30)
    port_eval_df_usd.loc['annual_sharpe'] = port_eval_df_usd.loc['annualized_returns']/port_eval_df_usd.loc['annual_vol']
    port_eval_df_usd.loc['mthly_sharpe'] = port_eval_df_usd.loc['mthly_returns']/port_eval_df_usd.loc['mthly_vol']
        
    
    port_eval_df_btc = pd.DataFrame(index=btc_metrics, columns=columns)
    port_eval_df_btc.loc['initial_capital'] = final_backtest_df['portfolio_balance_btc'].iloc[0]
    port_eval_df_btc.loc['cumulative_returns'] = final_backtest_df['portfolio_cum_returns_btc'].iloc[-1]
    port_eval_df_btc.loc['annualized_returns'] = final_backtest_df['portfolio_returns_btc'].mean() *365  #Crypto trades 365 days per year
    port_eval_df_btc.loc['mthly_returns'] = final_backtest_df['portfolio_returns_btc'].mean() *30
    port_eval_df_btc.loc['annual_vol'] = final_backtest_df['portfolio_returns_btc'].mean() * np.sqrt(365) #Crypto trades 365 days per year
    port_eval_df_btc.loc['mthly_vol'] = final_backtest_df['portfolio_returns_btc'].mean() * np.sqrt(30)
    port_eval_df_btc.loc['annual_sharpe'] = port_eval_df_btc.loc['annualized_returns']/port_eval_df_btc.loc['annual_vol']
    port_eval_df_btc.loc['mthly_sharpe'] = port_eval_df_btc.loc['mthly_returns']/port_eval_df_btc.loc['mthly_vol']
    
    # Calculate Sortino Ratios
    sortino_ratio_df = final_backtest_df[['portfolio_returns_usd','portfolio_returns_btc']].copy()
    sortino_ratio_df.loc[:, "downside_usd"] = 0
    sortino_ratio_df.loc[:,"downside_btc"] = 0
    target = 0
    mask_usd = sortino_ratio_df['portfolio_returns_usd'] < target
    mask_btc = sortino_ratio_df['portfolio_returns_btc'] < target
    
    sortino_ratio_df.loc[mask_usd, "portfolio_returns_usd"] = (sortino_ratio_df["portfolio_returns_usd"]**2)
    sortino_ratio_df.loc[mask_btc, "portfolio_returns_btc"] = (sortino_ratio_df["portfolio_returns_btc"]**2)
    
    down_stdev_usd_annual = np.sqrt(sortino_ratio_df['downside_usd'].mean()) * np.sqrt(365)
    down_stdev_btc_annual = np.sqrt(sortino_ratio_df['downside_btc'].mean()) * np.sqrt(365)
    down_stdev_btc_mthly = np.sqrt(sortino_ratio_df['downside_btc'].mean()) * np.sqrt(30)
    down_stdev_usd_mthly = np.sqrt(sortino_ratio_df['downside_usd'].mean()) * np.sqrt(30)
    
    expected_return_usd_annual = sortino_ratio_df['portfolio_returns_usd'].mean()*365
    expected_return_btc_annual = sortino_ratio_df['portfolio_returns_btc'].mean()*365
    sortino_ratio_usd_annual = expected_return_usd_annual/down_stdev_usd_annual
    sortino_ratio_btc_annual = expected_return_btc_annual/down_stdev_btc_annual
    
    expected_return_usd_mthly = sortino_ratio_df['portfolio_returns_usd'].mean()*30
    expected_return_btc_mthly = sortino_ratio_df['portfolio_returns_btc'].mean()*30
    
    sortino_ratio_usd_mthly = expected_return_usd_mthly/down_stdev_usd_mthly
    sortino_ratio_btc_mthly = expected_return_btc_mthly/down_stdev_btc_mthly
    
    port_eval_df_usd.loc['annual_sortino'] = sortino_ratio_usd_annual
    port_eval_df_usd.loc['mthly_sortino'] = sortino_ratio_usd_mthly
    port_eval_df_btc.loc['annual_sortino'] = sortino_ratio_btc_annual
    port_eval_df_btc.loc['mthly_sortino'] = sortino_ratio_usd_mthly
    
    
    
    
    # --------------------------------------------- print Performance stats ------------------------------------------
    # Print out Performance Tables
    print(port_eval_df_usd)
    print(port_eval_df_btc)
    
    # Risk Visualizations    
    #final_backtest_df['rolling_sharpe'] !!!! NEED TO MAKE SURE THERE IS A ROW EVERY 6 HOURS
    final_backtest_df[['portfolio_returns_usd', 'portfolio_returns_btc']]
    # Reward Measures
    final_backtest_df['portfolio_cum_returns_usd'].plot()
    final_backtest_df['portfolio_cum_returns_btc'].plot()  
    final_backtest_df['portfolio_returns_usd'].plot()
    final_backtest_df['portfolio_returns_btc'].plot()
    

### Team Tasks
- need test_df function to come in sequentially
- make sure test dataframe has rows that print every 6 hours ... note that shrimpy doesn't print candle if there is no tick. Do this in 
- finish strategy logic (update balances as we iterate through rows)

- check for balance calcs, time calc, and kpi calc errors
- return print out format like metin's return function
- 

#### Portfolio Backtest Performance Dashboard Model
    1. Input Variables
    2. Run Backtest
    3. Evaluate Static Performance Statistics
        - performance measured in usd
        - performance measured in btc
    4. Visualize Risk Adjusted Performance over time
        - Reward
            - Daily Returns
            - Cumulative Returns
            - Rolling Returns
            - Alpha
        - Risk
            - box plots
            - rolling vol
            - rolling sharpe
    
---

---

### 5. Live Trade Functions

In [ ]:
# ----------------------------- LIVE TRADING ----------------------------------------------
# --- copyright- (MANTIS TRADING, LLP // Mantis Copyright Token // open source // ) 
# --- mantistrading.win
# --- version "0.01"
# --- live_trader_mantis.py
# -----------------------------------------------------------------------------------------


def initialize():
# -------------------------------------------------------------------------------------------
# --------------- Initialize the dashboard, data storage, and account balances --------------
# -------------------------------------------------------------------------------------------
    print("Intializing Account and DataFrame")
    df = fetch_data()
    # DEFINE User ID
    shrimpy_users = client.list_users()
    shrimpy_user_id = users[0]['id'] # Could go crazy if we can write a scaling function here...multiplied with an AWS RoboAdivsor or app
    # Initialize Account
    account = {"balance": , "": 0}
    df = fetch_data()
    return account, df

def fetch_data():
    # Set environment variables from the .env file
    env_path = Path("/Users/gdepa")/'grant_api_keys.env'
    load_dotenv(env_path)
    shrimpy_public_key = os.getenv("SHRIMPY_PUBLIC_KEY")
    shrimpy_private_key = os.getenv("SHRIMPY_PRIVATE_KEY")

    shrimpy_client = shrimpy.ShrimpyApiClient(shrimpy_public_key, shrimpy_private_key)
    ## Retrieve price information from shrimpy websocket
    
    return prices_df

def generate_features(prices_df):
    ## Calcluate Feature Datframe from prices df
    return features_df
    
    
def rebalancePortfolio(features_df, account_id, user_id):
    # 
    return account




### May not need
async def main():
    loop = asyncio.get_event_loop()
    while True:
        global account
        global prices_df
        # global dashboard
        
        new_df = await loop.run_in_executor(Non, fetch_data)
        df = df.append(new_df, ignore_index=True)
        
        min_window = 22
        if df.shape[0]>= min_window:
            features = generate_features(prices_df)
            account = rebalancePortfolio(features, account)
            
        # update_dashboard(prices_df, dashboard)
        await asyncio.sleep(1)
        
# Python 3.7+
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


---

### 6. Real-Time Functions

In [ ]:
# ----------------------------- REAL-TIME MANAGER ---------------------------------
# --- copyright- (MANTIS TRADING, LLP // Mantis Copyright Token // open source // )
# --- mantistrading.win *
# --- version "0.01"
# --- real_time_manager.py
# ---------------------------------------------------------------------------------



# ----------------------- POPULATE Performance Dash -------------------------------
# Global Stats
    # Total Account Value
    # Capital at Risk: [BTC, %], [USD, %]  * exposure outside base currency
    # Expected Accumulation: [ BTC, % ], [USD,%]
    # Current Accuumulation: [ BTC, % ], [USD,%]

# Visualizations
    # (Multi and single account) Timeseries Options
        # BTC Accumulation relative to expected return
        # Bucket Allocation Weights relative to thresholds
        # ALT Asset Allocation weights relative to thresholds
 
    # (Multi and single account) Pie charts Options
        # Bucket Allocation weights relative to thresholds
        # ALT Asset Allocation weights relative to thresholds

    # (Multi and Single account) Radial
        # btc accumulation rate
        # bucket accumulation rates
        # ALT Asset Accumulation rates
    
# ------------------------------ Send Alerts --------------------------------------
# Send Performance Report every 6hrs
# Send Alert on RISKS Thresholds- [High negative accumulation_rate, max_drawdown, VAR, etc..]
# Send Alert on WINS-[High positive accumulation_rate]
# Send Alert on KILL SWITCH


# ---------------------------- Log Essential Data ----------------------------------
# 1. Save Trades DF to database
# 2. Save orders DF to database
# 3. Save capital_gains df to database
# 4. Save KPIs dataframe to database

---